In [23]:
import random
from abc import abstractmethod
from math import log
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from data_extaction import read_docs
from positional_indexing import PositionalIndexer
from preprocess import EnglishPreprocessor
import pprint
from query import calc_tfidf

from sklearn import svm

train_docs = read_docs('../data/phase2_train.csv')
test_docs = read_docs('../data/phase2_test.csv')

preprocessor = EnglishPreprocessor(train_docs)

for doc in train_docs.values():
    doc.words = preprocessor.preprocess(doc.text)
for doc in test_docs.values():
    doc.words = preprocessor.preprocess(doc.text)

print("Preprocess is done!")

index = PositionalIndexer(train_docs, 1).index
print("Index Created Successfully!")

Preprocess is done!
Index Created Successfully!


In [47]:
docs = [" ".join(doc.words) for doc in list(train_docs.values())]
size = len(docs)
train_size = int(0.9*size)
t_docs = docs[:train_size]
v_docs = docs[train_size:]

cv = CountVectorizer()
word_count_vector = cv.fit_transform(t_docs)

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

# count matrix
count_vector=cv.transform(t_docs)
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(tf_idf_vector.todense(), columns=feature_names, index=[i for i in range(train_size)])

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(df,[doc.tag for doc in list(train_docs.values())][:train_size])

predictions_SVM = SVM.predict(Test_X_Tfidf)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)